# Índice

1. [Introducción a Random Forest](#introducción-a-random-forest)

    1.0. [¿Qué es Random Forest?](#qué-es-random-forest)
    1.1. [Fundamentos teóricos](#fundamentos-teóricos)
    1.2. [Proceso de predicción: Votación y Promediado](#proceso-de-predicción-votación-y-promediado)
    1.3. [Ventajas y Desventajas](#ventajas-y-desventajas)

2. [Implementación del modelo](#implementación-del-modelo)
   2.1. [Packages](#packages)

   2.3. [Balancear mejor la muestra](#balancear-mejor-la-muestra)

# Introducción a Random Forest

## ¿Qué es Random Forest?

Dentro del vasto ecosistema de algoritmos de machine learning, el algoritmo Random Forest (o Bosque Aleatorio) se ha consolidado como una de las herramientas más poderosas y versátiles. Pertenece a la categoría de métodos de aprendizaje de conjunto (ensemble learning), que combinan las predicciones de múltiples modelos para producir un resultado más preciso y robusto que el de cualquier modelo individual.

Random Forest se construye a partir de una multitud de árboles de decisión, que son en sí mismos un tipo de algoritmo de aprendizaje supervisado. La metáfora del "bosque" es apropiada: así como un bosque está compuesto por muchos árboles, este algoritmo combina las predicciones de muchos árboles de decisión individuales para formar una predicción colectiva más fuerte y confiable.

Los árboles de decisión son algoritmos de aprendizaje supervisado que particionan recursivamente el espacio de características en regiones más pequeñas, asignando una etiqueta de clase o valor a cada región. Aunque son intuitivos y fáciles de interpretar, los árboles individuales tienden a sufrir de overfitting, especialmente con datos complejos.

## Fundamentos teóricos: 

**Bagging y Aleatoriedad de Características**

El poder de Random Forest reside en dos principios fundamentales.

- **Bagging (Bootstrap Aggregation)**: Introducido por Leo Breiman en 1996. Este método consiste en crear  múltiples subconjuntos de datos de entrenamiento mediante muestreo aleatorias con reemplazodel conjunto de datos original. Cada árbol se entrena con uno de estos subconjuntos.
- **Aleatoriedad de las Características**: En cada división de nodo, se considera solo un subconjunto aleatorio de características. Esto reduce la correlación entre los árboles. Cada subconjunto puede contener algunas observaciones repetidas y otras ausentes, lo que garantiza que cada árbol individual se entrene con una perspectiva ligeramente diferente de los datos. Al final, las predicciones de todos los árboles se combinan mediante votación mayoritaria (clasificación) o promediado (regresión).

## Proceso de predicción: Votación y Promediado

Para evitar que los árboles individuales se correlacionen demasiado (lo que ocurriría si todos los árboles se dividieran en las mismas características dominantes), Random Forest introduce una segunda capa de aleatoriedad. En cada nodo de cada árbol, el algoritmo no considera todas las características disponibles, sino que selecciona al azar un subconjunto de ellas para encontrar la mejor división.

- **Clasificación**: Cada árbol vota por una clase y la clase mayoritaria se selecciona como predicción.
- 
- **Regresión**: Se promedian las predicciones de todos los árboles.

Esta doble aleatorización (en las muestras y en las características) es lo que hace que Random Forest sea tan efectivo para reducir la varianza y mejorar la generalización.


## Ventajas y Desventajas

| Ventajas | Desventajas |
|----------|-------------|
| Alta precisión | Complejidad computacional |
| Robustez ante ruido y valores atípicos | Requiere más recursos |
| Maneja datos de alta dimensión | Menor interpretabilidad ("caja negra") |
| Flexibilidad (clasificación y regresión) | Período de entrenamiento largo |
| No requiere escalamiento de características | Puede tender a sobreajustar si no se sintoniza |
| Puede manejar valores faltantes | Tiende a tener un sesgo hacia variables con muchos valores únicos |

# Implementación del modelo

## Packages

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath(".."))

from mlparadetectarfraudes.data import data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.feature_selection import SelectFromModel

# Importar datos estandarizados

In [ ]:
#data_estandarizados = pd.read_csv(data_interim_dir("data_estandarizados.csv"))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Importancia de características
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
feature_importances.nlargest(10).plot(kind='barh')
plt.title('Top 10 Importancia de Características')
plt.show()
# Visualización de un árbol individual (requiere graphviz)
from sklearn.tree import plot_tree
plt.figure(figsize=(20,10))
plot_tree(rf_model.estimators_[0], feature_names=X.columns, filled=True, rounded=True, max_depth=3)
plt.title("Ejemplo de Árbol de Decisión en el Bosque")
plt.show()

In [ ]:
### . Optimización de Hiperparámetros

from sklearn.model_selection import GridSearchCV
# Definir la grilla de parámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}
# Búsqueda en grilla
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)
# Mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)


In [ ]:
# Separar features y target
x = data.drop('is_fraud', axis=1)
y = data['is_fraud']

# Balancear datos con SMOTE
smote = SMOTE(sampling_strategy=0.3, random_state=42)
x_res, y_res = smote.fit_resample(x, y)

# Split
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.2, random_state=42, stratify=y_res)

# Entrenamiento
model_RF = RandomForestClassifier(
    n_estimators=100,    # Número de árboles en el bosque
    criterion='gini',    # Función para medir la calidad de la división
    max_depth=100,      # Profundidad máxima de los árboles
    min_samples_split=2, # Mínimo número de muestras requeridas para dividir un nodo interno
    min_samples_leaf=1,  # Mínimo número de muestras requeridas en un nodo hoja
    max_features='auto', # Número de características a considerar para buscar la mejor división
    bootstrap=True,      # Whether to use bootstrap samples when building trees
    random_state=42,     # Semilla para reproducibilidad
    class_weight='balanced',  # Ajustar pesos para clases desbalanceadas
    n_jobs=1               # Número de trabajos a ejecutar en paralelo
)
model_RF.fit(x_train, y_train)

# Evaluación
y_pred = model_RF.predict(x_test)
y_pred_proba = model_RF.predict_proba(x_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.4f}")
print('Reporte de clasificación')
print(classification_report(y_test, y_pred))

print("Random Forest - Rendimiento:")
print(classification_report(y_test, y_pred))
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")

# Matriz de confusión
print('Matriz de confusión')
print(confusion_matrix(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Random Forest')
plt.ylabel('Verdaderos')
plt.xlabel('Predichos')
plt.show()

# Asegúrate de que la carpeta exista
os.makedirs('../models', exist_ok=True)

# Guarda los archivos en la carpeta models
joblib.dump(model_RF, '../models/model_fraude_RF.pkl')
joblib.dump(scaler, '../models/scaler.pkl')  # Save the scaler as well

# Función de predicción
def predict_fraude(transaction_data: dict, model):
    # Create DataFrame from input
    transaction_data_df = pd.DataFrame(transaction_data, index=[0])

    # Scale numerical features
    transaction_data_df[['amount', 'user_age']] = scaler.transform(transaction_data_df[['amount', 'user_age']])

    # One-hot encode categorical features
    transaction_data_df = pd.get_dummies(transaction_data_df)

    # Ensure all columns are present
    for col in x_train.columns:
        if col not in transaction_data_df.columns:
            transaction_data_df[col] = 0

    # Reorder columns to match training data
    transaction_data_df = transaction_data_df[x_train.columns]

    prob = model.predict_proba(transaction_data_df)[0][1]  # Fixed typo: predict_prob to predict_proba
    return {'fraud_probability': round(prob, 4), 'is_fraud': prob >= 0.5}

In [ ]:
# 2. Entrenar
pipeline.fit(X_train, y_train)

# 3. Predecir sobre registro nuevo
new_tx = {'amount': 123.45, 'user_age': 35, 'country': 'US', …}
new_df = pd.DataFrame([new_tx])

prob = pipeline.predict_proba(new_df)[0, 1]   # Probabilidad de fraude
label = pipeline.predict(new_df)[0]           # 0 o 1

print({'prob_fraude': prob, 'alertra': bool(label)})

In [ ]:
# Interpretación del modelo: Importancia de características
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Obtener la importancia de las características
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Visualizar las características más importantes
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importances.head(15))
plt.title('Top 15 Características Más Importantes para Detección de Fraudes')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

# Análisis de las probabilidades predichas
fraud_probas = y_pred_proba[y_test == 1]  # Probabilidades para casos de fraude reales
non_fraud_probas = y_pred_proba[y_test == 0]  # Probabilidades para casos normales

plt.figure(figsize=(10, 6))
plt.hist(non_fraud_probas, bins=50, alpha=0.7, label='Transacciones Normales')
plt.hist(fraud_probas, bins=50, alpha=0.7, label='Fraudes Reales')
plt.xlabel('Probabilidad Predicha de Fraude')
plt.ylabel('Frecuencia')
plt.title('Distribución de Probabilidades Predichas')
plt.legend()
plt.show()

## Optimización de Hiperparámetros

In [ ]:
# Optimización de hiperparámetros con GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Definir la métrica de evaluación (F1-score es importante para datos desbalanceados)
scorer = make_scorer(f1_score, average='weighted')

# Definir la grilla de parámetros a probar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Configurar la búsqueda en grilla
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, class_weight='balanced'),
    param_grid=param_grid,
    scoring=scorer,
    cv=5,           # Validación cruzada de 5 folds
    n_jobs=-1,      # Usar todos los procesadores disponibles
    verbose=1
)

# Ejecutar la búsqueda (puede tomar tiempo)
grid_search.fit(X_train, y_train)

# Mejores parámetros encontrados
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación F1:", grid_search.best_score_)

# Entrenar el modelo final con los mejores parámetros
best_rf_model = grid_search.best_estimator_

# Balancear mejor la muestra

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Opción 1: Usar SMOTE para oversampling de la clase minoritaria
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print("Distribución después de SMOTE:")
print(pd.Series(y_resampled).value_counts())

# Opción 2: Combinar oversampling y undersampling
resampling_pipeline = ImbPipeline([
    ('oversample', SMOTE(sampling_strategy=0.1, random_state=42)),
    ('undersample', RandomUnderSampler(sampling_strategy=0.5, random_state=42))
])

X_resampled, y_resampled = resampling_pipeline.fit_resample(X_train, y_train)

# Entrenar Random Forest con datos balanceados
rf_balanced = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'  # Esto da más peso a la clase minoritaria
)
rf_balanced.fit(X_resampled, y_resampled)

# Predecir y evaluar
y_pred_balanced = rf_balanced.predict(X_test)
y_pred_proba_balanced = rf_balanced.predict_proba(X_test)[:, 1]

print("Random Forest Balanceado - Rendimiento:")
print(classification_report(y_test, y_pred_balanced))
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba_balanced):.4f}")

# Matriz de confusión para Random Forest balanceado
cm_rf_balanced = confusion_matrix(y_test, y_pred_balanced)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf_balanced, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Fraude', 'Fraude'],
            yticklabels=['No Fraude', 'Fraude'])
plt.title('Matriz de Confusión - Random Forest Balanceado')
plt.ylabel('Verdaderos')
plt.xlabel('Predichos')
plt.show()

# Model pipeline

In [ ]:
# Separate features and target
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']

# Identify column types
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create preprocessing pipelines
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), #Reemplazar valores faltantes con la mediana
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), #Reemplazar valores faltantes con  los valores más comunes
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first')) # Use variables dummies
])

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Example of how the final pipeline will look
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(sampling_strategy=0.3, random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])